In [1]:
from meta_processing import *
import matplotlib 
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ms_plotter_tools as msp
matplotlib.rcParams['font.family'] = 'serif'

In [2]:
# TO DO - SET UP FILE FOR BELOW PARAMS

# directory containing spectra as .d folders (unzipped). Remember to add double \\ in path

# directory = "D:\\charlie HRMS data\\PDRA Ocr23-Jan24 data\\20240123\\4 dC"
# directory = "D:\\WORK\\20230710 DOE_CTB H6_5M_DAmp\\testing"
directory = "D:\\mass spectra\\DoE experiment"

# unzip directory if required - deletes zip file after
unzip_from_dir(directory)

# scan window in TIC
scanstart = 427
scanend = 502

# mass window of deconvolution
masslb = 10000
massub = 17000

# m/z window of spectra to be deconvolved
minmz = 600

# number of iterations of deconvolution algorithm (recommended 50-100)
numit = 100

# charges 
startz = 1
endz = 100
numz = 100

# expected masses
# masslist = [11814, 12488, 12417, 12254]
masslist = [13597.42, 13151.97, 12559.36]



# names of expected masses
# names = ["GGG-CTB", "AYLPETGGG-CTB", "YLPETGGG-CTB", "LPETGGG-CTB"]
names = ["CTB-H6", "CTB-LPETGVSEYG", "CTB-LPET"]

tolerance = 10

In [ ]:
# load engine
eng = Meta2()

# extract scans from baf
eng.upload_spectra(directory, scanstart, scanend)

# create hdf5 file
eng.load_hdf5(directory)

# load data to unidec & hdf5 file 
eng.to_unidec()

# set parameters, n.b. further paramters can be set in self.eng.config (## to do - add config file upload)
eng.update_config(massub = massub,
                 masslb = masslb,
                 minmz = minmz,
                 startz = startz,
                 endz = endz, 
                 numz = numz)

# run unidec deconvolution
eng.on_unidec()

# match peaks in deconvolution to expected masses. Background = True finds peaks above noise. 
eng.match_spectra(masslist, names, tolerance, background = True)

# export data to excel 
eng.export_data(export=True)


UniDec Engine v.6.0.4

UniDec Path: C:\Users\cm19ljc\Anaconda3\envs\unidec3\lib\site-packages\unidec\bin\unidec.exe
Specified BAF has 739 spectra with line and profile data.
Specified BAF has 744 spectra with line and profile data.
Specified BAF has 744 spectra with line and profile data.
Specified BAF has 744 spectra with line and profile data.
Specified BAF has 739 spectra with line and profile data.
Specified BAF has 739 spectra with line and profile data.
Specified BAF has 730 spectra with line and profile data.
Specified BAF has 739 spectra with line and profile data.
Specified BAF has 738 spectra with line and profile data.
Adding: 240126 DoE Nexp 1_1-2-33_1_8675.d to ms_dataset
Adding: 240126 DoE Nexp 2_1-2-34_1_8676.d to ms_dataset
Adding: 240126 DoE Nexp 3_1-2-35_1_8677.d to ms_dataset
Adding: 240126 DoE Nexp 4_1-2-36_1_8678.d to ms_dataset
Adding: 240126 DoE Nexp 5_3-1-40_1_8708.d to ms_dataset
Adding: 240126 DoE Nexp 6_3-1-41_1_8709.d to ms_dataset
Adding: 240126 DoE Nexp 7

In [ ]:
df = eng.export_data(export=False)
df.set_index('Name', inplace = True)
df_pc = df['Percentage_Labelling'].reset_index()
df_pc

In [ ]:
# import ms_plotter_tools as msp

spectra = eng.eng.data.spectra

# add in zoomed function that plots over window using min and max masses contained in masslist 
msp.plot_spectra_separate(spectra, attr = "massdat", xlabel = "Mass [Da]", 
                          export=False, c='black',lw=0.7,window=[12000, 14000]
                         show_peaks=True)

# plot_spectra_separate(spectra, attr = "data2", xlabel = "m/z", export=False)



    

In [ ]:
spectra = eng.eng.data.spectra


alpha = np.linspace(1, 0.6, len(spectra))

gray_cmap = msp.get_cmap(len(spectra), cmap = "gray", x1 = 0, x2 = 0.7)

msp._set_spectra_colors(spectra, cmap ="rainbow" )
fig = plt.figure()

for i,s in enumerate(spectra):
    xoff = i/+5
    yoff= i/10+0.05
    axs=fig.add_axes([xoff,yoff,0.65,0.65], zorder=-i)
    axs.plot(s.massdat[:, 0], s.massdat[:, 1], color = gray_cmap[i], alpha = alpha[i])
    axs.set_title(s.name, alpha = alpha[i], backgroundcolor = 'white')
    axs.spines['right'].set_visible(False)
    axs.spines['top'].set_visible(False)
    axs.spines['left'].set_visible(False)
    axs.yaxis.set_tick_params(labelleft=False, )
    axs.set_yticks([])
    axs.set_clip_on(False)
    axs.grid(False)
    axs.set_xlim(10000, 17000)     
    axs.spines['bottom'].set_color(gray_cmap[i])
    axs.tick_params(axis='x', colors=gray_cmap[i])

#         axs.set_facecolor(None)

    if i != 0:
#             axs.spines['bottom'].set_visible(False)
        axs.xaxis.set_tick_params(labelleft=False)
#             axs.set_xticks([])

#         sns.despine(offset=10)
    axs.patch.set_alpha(0)
        
plt.show()